In [3]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 49766334464 (46.35 GB)
Present Capacity: 32830218240 (30.58 GB)
DFS Remaining: 32830169088 (30.58 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.2:9866 (p4_whagedorn-dn-1.p4_whagedorn_default)
Hostname: 2a41e07ddd5d
Decommission Status : Normal
Configured Capacity: 24883167232 (23.17 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 8451280896 (7.87 GB)
DFS Remaining: 16415084544 (15.29 GB)
DFS Used%: 0.00%
DFS Remaining%: 65.97%
Configured Cache

In [4]:
import os
import pyarrow.csv as pa
import pyarrow.parquet as pq

csv_url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
csvPath = "/nb/hdma-wi-2021.csv"
parquetPath = "/nb/hdma-wi-2021.parquet"

if not os.path.exists(csvPath):
    !wget -O {csvPath} {csv_url}

df = pa.read_csv(csvPath)
pq.write_table(df, parquetPath)

In [5]:
# Removes existing files
!hdfs dfs -rm -f hdfs://boss:9000/single.parquet
!hdfs dfs -rm -f hdfs://boss:9000/double.parquet

# New files
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp file:///nb/hdma-wi-2021.parquet hdfs://boss:9000/single.parquet
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp file:///nb/hdma-wi-2021.parquet hdfs://boss:9000/double.parquet

2024-10-21 02:35:13,931 INFO hdfs.DFSOutputStream: Exception while adding a block
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.NotReplicatedYetException): Not replicated yet: /single.parquet._COPYING_
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.validateAddBlock(FSDirWriteFileOp.java:181)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getAdditionalBlock(FSNamesystem.java:2977)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.addBlock(NameNodeRpcServer.java:912)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.addBlock(ClientNamenodeProtocolServerSideTranslatorPB.java:595)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:621)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcI

In [6]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/double.parquet
!hdfs dfs -du -h hdfs://boss:9000/single.parquet

15.9 M  31.7 M  hdfs://boss:9000/double.parquet
15.9 M  15.9 M  hdfs://boss:9000/single.parquet


In [7]:
#q3
import requests

res = requests.get("http://boss:9870/webhdfs/v1/single.parquet?op=GETFILESTATUS")
fileStatus = res.json()
fileStatus

{'FileStatus': {'accessTime': 1729478113248,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 16642976,
  'modificationTime': 1729478115315,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [8]:
#q4
res = requests.get("http://boss:9870/webhdfs/v1/single.parquet?op=OPEN&offset=0&noredirect=true")
loc = res.json()["Location"]
loc

'http://2a41e07ddd5d:9864/webhdfs/v1/single.parquet?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [9]:
#q5
length = fileStatus["FileStatus"]["length"]
blockSize = fileStatus["FileStatus"]["blockSize"]
numBlocks = (length + blockSize - 1) // blockSize

dict = {}
for i in range(numBlocks):
    off = i * blockSize
    res = requests.get(f"http://boss:9870/webhdfs/v1/single.parquet?op=OPEN&offset={off}&noredirect=true")
    loc = res.json()["Location"]

    id = loc.split("//")[1].split(":")[0]
    if id in dict:
        dict[id] += 1
    else:
        dict[id] = 1

dict

{'2a41e07ddd5d': 6, 'c1a5d924c17a': 10}

In [10]:
#q6
res = requests.get("http://boss:9870/webhdfs/v1/double.parquet?op=GETFILEBLOCKLOCATIONS")
locations = res.json()["BlockLocations"]["BlockLocation"]

dict = {}
for i in locations:
    host1 = i["hosts"][0]
    host2 = i["hosts"][1]

    if host1 in dict:
        dict[host1] += 1
    else:
        dict[host1] = 1

    if host2 in dict:
        dict[host2] += 1
    else:
        dict[host2] = 1

dict

{'c1a5d924c17a': 16, '2a41e07ddd5d': 16}

In [22]:
#q7
import pyarrow as pa
import pyarrow.fs
import time
import pyarrow.compute as pc

hdfs = pa.fs.HadoopFileSystem("boss", 9000)

start = time.time()

with hdfs.open_input_file("/double.parquet") as f:
    t = pa.parquet.read_table(f)

avgLoan = pc.mean(t["loan_amount"])

end = time.time()

totalTime1 = end - start

avgLoan, totalTime1

(<pyarrow.DoubleScalar: 204961.21752386744>, 0.7812724113464355)

In [23]:
#q8
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

start = time.time()

with hdfs.open_input_file("/double.parquet") as f:
    t = pa.parquet.read_table(f, columns = ["loan_amount"])

avgLoan = pc.mean(t["loan_amount"])

end = time.time()

totalTime2 = end - start

diff = totalTime1 / totalTime2

diff

12.956195808177258